## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



1) Get familiar with data

2) Prepare the data for processing

3) Make action plan

4) Develop the code that works with a sample

5) Adjust the code to work with Big data

6) Test big data approach on a sample

7) Run program with big data

## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

In [102]:
%%time
#(33, 1299993)
#CPU times: user 9min 20s, sys: 48.3 s, total: 10min 8s
#Wall time: 10min 10s

import pandas as pd
#path='/home/dsc/Data/challenge'
path='../data/challenge'
#file_name='bookings.sample.csv.bz2'
file_name='bookings.csv.bz2'
filepath=path+'/'+file_name



bc=pd.read_csv(filepath,sep='^', chunksize=300000, low_memory=False)

all_chunks=pd.DataFrame()
for i,chunk in enumerate(bc):
    all_chunks=all_chunks.append(chunk)
    all_chunks=all_chunks.drop_duplicates()
    print (i, len(all_chunks))

0 300000
1 600000
2 900000
3 1000000
4 1000000
5 1000000
6 1000000
7 1000000
8 1000000
9 1000000
10 1000000
11 1000000
12 1000000
13 1000000
14 1000000
15 1000000
16 1299993
17 1299993
18 1299993
19 1299993
20 1299993
21 1299993
22 1299993
23 1299993
24 1299993
25 1299993
26 1299993
27 1299993
28 1299993
29 1299993
30 1299993
31 1299993
32 1299993
33 1299993
CPU times: user 7min 2s, sys: 1min 2s, total: 8min 5s
Wall time: 8min 8s


[StackOverFlow about LowMemory](http://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options)

In [103]:
all_chunks.to_csv('bookings_wn_dup.csv',sep='^',index=False)
! bzip2 -f bookings_wn_dup.csv
#Why dont we remove the blank spaces of columns here? we could but... are we sure noone else is using this input?
#We would break his app with this change.... so lets just remove the duplicates and dont change the structure
bk_no_dup=pd.read_csv('bookings_wn_dup.csv.bz2',sep='^', low_memory=False)
#without low memory False the column type is set with respect to first N rows
bk_no_dup.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1.0,2013.0,3.0,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1.0,2013.0,3.0,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1.0,2013.0,3.0,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1.0,2013.0,3.0,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1.0,2013.0,3.0,SYDA82546


In [104]:
! bzcat bookings_wn_dup.csv.bz2 | wc -l

 1299994


#### We have seen that we have white space in some columns....

In [6]:
bk_no_dup.size

49399734

In [7]:
all_chunks.columns.tolist()
#list(all_chunks.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [8]:
all_chunks.columns=all_chunks.columns.map(lambda x:str(x).strip())

In [9]:
all_chunks.columns.tolist()

['act_date',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid',
 'rloc',
 'cre_date',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time',
 'off_time',
 'pax',
 'year',
 'month',
 'oid']

In [10]:
b=all_chunks[['cre_date','act_date', 'dep_port', 'arr_port']].copy()
#si es un round trip lo vamos a tener 2 veces

In [11]:
#b.head()
b.iloc[0].tolist()

['2013-02-22 00:00:00', '2013-03-05 00:00:00', 'ZRH     ', 'LHR     ']

In [12]:
b.dep_port=b.dep_port.map(lambda x:str(x).strip())
b.arr_port=b.arr_port.map(lambda x:str(x).strip())
##b['arr_port']=b['arr_port'].map(lambda x: x.strip())

In [13]:
#b.head()
b.iloc[0].tolist()

['2013-02-22 00:00:00', '2013-03-05 00:00:00', 'ZRH', 'LHR']

###  Search

In [14]:
%%time
#12 min
#(67, 718003)
#CPU times: user 10min 29s, sys: 1min 2s, total: 11min 32s
#Wall time: 11min 32s
    
import pandas as pd
#path='/home/dsc/Data/challenge'
path='../data/challenge'
#file_name='searches.sample.csv.bz2'
file_name='searches.csv.bz2'
filepath=path+'/'+file_name


all_chunks=pd.DataFrame()
bc=pd.read_csv(filepath,sep='^', chunksize=300000, low_memory=False)

for i,chunk in enumerate(bc):
    all_chunks=all_chunks.append(chunk)
    all_chunks=all_chunks.drop_duplicates()
    print (i, len(all_chunks))
    
    
    

0 299999
1 358999
2 358999
3 358999
4 658992
5 718002
6 718002
7 718002
8 718002
9 718002
10 718002
11 718002
12 718002
13 718002
14 718002
15 718002
16 718002
17 718002
18 718002
19 718002
20 718002
21 718002
22 718002
23 718002
24 718002
25 718002
26 718002
27 718002
28 718002
29 718002
30 718002
31 718002
32 718002
33 718002
34 718002
35 718002
36 718002
37 718002
38 718002
39 718002
40 718002
41 718002
42 718002
43 718002
44 718002
45 718002
46 718002
47 718002
48 718002
49 718002
50 718002
51 718002
52 718002
53 718002
54 718002
55 718002
56 718002
57 718002
58 718002
59 718002
60 718002
61 718002
62 718002
63 718002
64 718002
65 718002
66 718002
67 718003
CPU times: user 9min 40s, sys: 1min 42s, total: 11min 23s
Wall time: 11min 28s


In [15]:
all_chunks.to_csv('searches_wn_dup.csv',sep='^', index=False)
! bzip2 -f searches_wn_dup.csv
sr_no_dup=pd.read_csv('searches_wn_dup.csv.bz2',sep='^', low_memory=False)
#without low memory False the column type is set with respect to first N rows
sr_no_dup.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


In [16]:
s=all_chunks[['Date', 'Origin','Destination']]

In [17]:
s.head()

,Date,Origin,Destination
0,2013-01-01,TXL,AUH
1,2013-01-01,ATH,MIL
2,2013-01-01,ICT,SFO
3,2013-01-01,RNB,ARN
4,2013-01-01,OSL,MAD


In [18]:
s.iloc[0].tolist()

['2013-01-01', 'TXL', 'AUH']

### NaN and prepare dates

In [31]:
print("size of Book:\t %d \nsize of Search:\t %d"% (len(b), len(s)))

size of Book:	 1299993 
size of Search:	 718003


In [32]:
b=b.dropna()

In [33]:
b['act_date']=pd.to_datetime(b['act_date'],errors='raise')

In [34]:
b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1299993 entries, 0 to 5099999
Data columns (total 4 columns):
cre_date    1299993 non-null object
act_date    1299993 non-null datetime64[ns]
dep_port    1299993 non-null object
arr_port    1299993 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 49.6+ MB


In [35]:
b.drop('cre_date', axis=1, inplace=True)
b.head()

,act_date,dep_port,arr_port
0,2013-03-05,ZRH,LHR
1,2013-03-26,SAL,CLT
2,2013-03-26,SAL,CLT
3,2013-03-26,AKL,SVO
4,2013-03-26,AKL,SVO


In [36]:
s=s.dropna()
s['Date'] = pd.to_datetime(s['Date'],errors='raise')

In [38]:
print("size of Book:\t%d \nsize of Search:\t%d " % (len(b), len(s)))

size of Book:	1299993 
size of Search:	718002 


## 3) Make action plan

Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

1) remove duplicates

2) parse dates from string to datetime

3) remove whitespaces
    a) from colum names
    b) from content
    
4) remove NaN

5) define the model

    if there is one booking for a given O&D done at the same day as the search (for a given O&D), ALL searches of the day (for a given 0&D) might have resulted from the same source and will be set with 1.
    This is regardless of the boarding time of the plane... So if I was looking for plane for the first 4 days of December for a given O&D all those searches would be set to 1 not just the one correspoding to the correct boarding time

        match
        Search : [search_date, O&D] 
        Booking: [Activity_date, O&D]
        
6) execute the model

    a) Group by bookings on [Activity_date, O&D] so that we dont have duplicates 
        (and we can have number of bookings for the day)
        or we can just drop the duplicates
        
    b) search left join bookings adding "Booked" column
    
    c) test if the merge was done right
    
    d) fill NaN of "booked" column with 0
    
    e) pull all values of booked column >1 to 1

more complex... get number of segments from searches
match search_date, and then split all O&D of all segments, and match the date of first flight of each segment (seg1Date, seg2Date)
with boarding time and O&D and act_date of booking

What do we have?

In [39]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [40]:
import pandas as pd

#path='/home/dsc/Data/challenge'
path='./'
file_nameb='bookings_wn_dup.csv.bz2'
file_names='searches_wn_dup.csv.bz2'

filepathb=path+'/'+file_nameb
filepaths=path+'/'+file_names

b=pd.read_csv(file_nameb,sep='^',usecols=['act_date           ','dep_port', 'arr_port'])
s=pd.read_csv(file_names,sep='^',usecols=['Date', 'Origin','Destination'])


In [41]:
print(len(b), len(s))
print("Booking size:%d \t\t Search size:%d" % (len(b),len(s)))
#s.tail()

1299993 718003
Booking size:1299993 		 Search size:718003


In [42]:
b.columns.tolist()

['act_date           ', 'dep_port', 'arr_port']

In [43]:
b.columns[0]

'act_date           '

In [44]:
b.columns.map(lambda x:str(x).strip())

Index(['act_date', 'dep_port', 'arr_port'], dtype='object')

In [45]:
b.head()

,act_date,dep_port,arr_port
0,2013-03-05 00:00:00,ZRH,LHR
1,2013-03-26 00:00:00,SAL,CLT
2,2013-03-26 00:00:00,SAL,CLT
3,2013-03-26 00:00:00,AKL,SVO
4,2013-03-26 00:00:00,AKL,SVO


In [46]:
b.iloc[0].tolist()

['2013-03-05 00:00:00', 'ZRH     ', 'LHR     ']

In [47]:
b.ix[0].tolist()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


['2013-03-05 00:00:00', 'ZRH     ', 'LHR     ']

In [48]:
b_smaller=b[b['dep_port']=="SAL     "]
b_smaller.head()

,act_date,dep_port,arr_port
1,2013-03-26 00:00:00,SAL,CLT
2,2013-03-26 00:00:00,SAL,CLT
2245,2013-03-12 00:00:00,SAL,LAX
2246,2013-03-12 00:00:00,SAL,LAX
2247,2013-03-14 00:00:00,SAL,LAX


In [49]:
b_smaller.iloc[3]

act_date               2013-03-12 00:00:00
dep_port                          SAL     
arr_port                          LAX     
Name: 2246, dtype: object

In [50]:
b_smaller.ix[2246]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


act_date               2013-03-12 00:00:00
dep_port                          SAL     
arr_port                          LAX     
Name: 2246, dtype: object

In [51]:
b.iloc[0]['dep_port'].strip()

'ZRH'

In [52]:
b.iloc[0]['arr_port'].strip()

'LHR'

In [53]:
pd.to_datetime(b.iloc[0]['act_date           '],errors='raise')

Timestamp('2013-03-05 00:00:00')

In [54]:
pd.to_datetime(b['act_date           '],errors='raise').head()

0   2013-03-05
1   2013-03-26
2   2013-03-26
3   2013-03-26
4   2013-03-26
Name: act_date           , dtype: datetime64[ns]

In [56]:
print("before drop", len(b))
b=b.dropna()
print("after drop", len(b))

b.columns=b.columns.map(lambda x:str(x).strip())
b.dep_port=b.dep_port.map(lambda x:str(x).strip())
b.arr_port=b.arr_port.map(lambda x:str(x).strip())
b['act_date']=pd.to_datetime(b['act_date'],errors='raise')
b.head()


before drop 1299993
after drop 1299993


,act_date,dep_port,arr_port
0,2013-03-05,ZRH,LHR
1,2013-03-26,SAL,CLT
2,2013-03-26,SAL,CLT
3,2013-03-26,AKL,SVO
4,2013-03-26,AKL,SVO


In [57]:
s.columns.tolist()

['Date', 'Origin', 'Destination']

In [58]:
s.iloc[0].tolist()

['2013-01-01', 'TXL', 'AUH']

In [59]:
s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718003 entries, 0 to 718002
Data columns (total 3 columns):
Date           718003 non-null object
Origin         718002 non-null object
Destination    718002 non-null object
dtypes: object(3)
memory usage: 16.4+ MB


In [61]:
print("before drop", len(s))
s=s.dropna()
print("after drop", len(s))
s['Date'] = pd.to_datetime(s['Date'],errors='raise')
s.head()
#print s.tail()

before drop 718003
after drop 718002


,Date,Origin,Destination
0,2013-01-01,TXL,AUH
1,2013-01-01,ATH,MIL
2,2013-01-01,ICT,SFO
3,2013-01-01,RNB,ARN
4,2013-01-01,OSL,MAD


In [47]:
#s.tail(10)

In [63]:
print("size of Book:\t%d \nsize of Search:\t%d " % (len(b), len(s)))

size of Book:	1299993 
size of Search:	718002 


In [64]:
b.iloc[0].tolist()

[Timestamp('2013-03-05 00:00:00'), 'ZRH', 'LHR']

In [65]:
s.iloc[0].tolist()

[Timestamp('2013-01-01 00:00:00'), 'TXL', 'AUH']

What do we have so far?

In [66]:
#s_all=pd.read_csv(file_names,sep='^',nrows=10)
s_all=pd.read_csv(file_names,sep='^')
pd.set_option('display.max_columns', None)
s_all.sample(2)


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
298413,2013-10-26,11:01:45,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,DUS,AGP,1.0,2.0,DUS,AGP,2014-09-22,BD,NaN,AGP,DUS,2014-10-02,BD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,0
679099,2013-03-26,22:33:41,MPT,a3ab256abb78596dad0f7a49091a9829,GB,YYC,MEL,0.0,1.0,YYC,MEL,2013-04-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LON


In [67]:
s_all.tail()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
717998,2013-05-03,10:28:25,MPT,01a4373dc0878961ab038a4089f8d4a7,DK,CPH,AYT,1.0,2.0,CPH,AYT,2013-08-03,NaN,NaN,AYT,CPH,2013-08-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,CPH
717999,2013-05-03,15:00:52,MPT,4e93208c7efe7d16455c6a887367cdbd,US,NYC,YTO,0.0,1.0,NYC,YTO,2013-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,DCA
718000,2013-05-03,00:40:36,FXP,439c0bb10ec8b58c68da567c40399a05,NO,TRF,TRD,0.0,1.0,TRF,TRD,2013-08-01,HP,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL
718001,2013-05-03,21:10:30,MPT,3561a60621de06ab1badc8ca55699ef3,US,IAH,RDU,1.0,2.0,IAH,RDU,2013-07-04,NaN,NaN,RDU,IAH,2013-07-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
718002,2013-10-13,18:57:54,MTP,e41c9d833aa74600552f2ed688b67d81,AT,VIE,HA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
len(s)

718002

In [69]:
import pandas as pd

#path='/home/dsc/Data/challenge'
path='.'
file_nameb='bookings_wn_dup.csv.bz2'
file_names='searches_wn_dup.csv.bz2'

filepathb=path+'/'+file_nameb
filepaths=path+'/'+file_names

b=pd.read_csv(file_nameb,sep='^',usecols=['act_date           ','dep_port', 'arr_port'])
s=pd.read_csv(file_names,sep='^',usecols=['Date', 'Origin','Destination'])

b=b.dropna()
b.columns=b.columns.map(lambda x:str(x).strip().upper())
b.dep_port=b.dep_port.map(lambda x:str(x).strip().upper())
b.arr_port=b.arr_port.map(lambda x:str(x).strip().upper())
b['act_date']=pd.to_datetime(b['act_date'],errors='raise')


s=s.dropna()
s['Date'] = pd.to_datetime(s['Date'],errors='raise')

Group by bookings so that we dont have duplicates, and we can have number of bookings

2nd option would be to simply drop duplicates
    b_no_dup=b.drop_duplicates()

In [70]:
#b_no_dup=b.drop_duplicates()

In [71]:
b_gr=b.groupby(['dep_port','arr_port', 'act_date'])

In [73]:
#uno es ida y otro es vuelta
for i, (group, values) in enumerate(b_gr):
    print(group)
    print(values)
    if (i==2) :break

('AAB', 'ROV', Timestamp('2013-05-29 00:00:00'))
         act_date dep_port arr_port
387078 2013-05-29      AAB      ROV
387079 2013-05-29      AAB      ROV
('AAB', 'ROV', Timestamp('2013-08-24 00:00:00'))
         act_date dep_port arr_port
572522 2013-08-24      AAB      ROV
572523 2013-08-24      AAB      ROV
('AAE', 'ALG', Timestamp('2013-01-07 00:00:00'))
         act_date dep_port arr_port
165186 2013-01-07      AAE      ALG


In [77]:
b_gr=b.groupby(['dep_port','arr_port', 'act_date']).size()
b_gr.head()

dep_port  arr_port  act_date  
AAB       ROV       2013-05-29    2
                    2013-08-24    2
AAE       ALG       2013-01-07    1
                    2013-01-08    2
                    2013-01-09    6
dtype: int64

In [88]:
b_gr=b.groupby(['dep_port','arr_port', 'act_date']).size().reset_index()
b_gr.head()

,dep_port,arr_port,act_date,0
0,AAB,ROV,2013-05-29,2
1,AAB,ROV,2013-08-24,2
2,AAE,ALG,2013-01-07,1
3,AAE,ALG,2013-01-08,2
4,AAE,ALG,2013-01-09,6


In [90]:
b_gr.rename(columns={0:'Num of Bookings'}, inplace=True)
print(len(b_gr))
b_gr.head()

466228


,dep_port,arr_port,act_date,Num of Bookings
0,AAB,ROV,2013-05-29,2
1,AAB,ROV,2013-08-24,2
2,AAE,ALG,2013-01-07,1
3,AAE,ALG,2013-01-08,2
4,AAE,ALG,2013-01-09,6


In [91]:
s.head()

,Date,Origin,Destination
0,2013-01-01,TXL,AUH
1,2013-01-01,ATH,MIL
2,2013-01-01,ICT,SFO
3,2013-01-01,RNB,ARN
4,2013-01-01,OSL,MAD


In [92]:
pd.merge?

In [92]:
s.tail()

,Date,Origin,Destination
717998,2013-05-03,CPH,AYT
717999,2013-05-03,NYC,YTO
718000,2013-05-03,TRF,TRD
718001,2013-05-03,IAH,RDU
718002,2013-10-13,VIE,HA


In [119]:
len(s) #it should be 718003 in order for index to correspond to number of row

718002

In [93]:
s_with_index=s.reset_index()
s_with_index.tail()

,index,Date,Origin,Destination
717997,717998,2013-05-03,CPH,AYT
717998,717999,2013-05-03,NYC,YTO
717999,718000,2013-05-03,TRF,TRD
718000,718001,2013-05-03,IAH,RDU
718001,718002,2013-10-13,VIE,HA


In [94]:
s_with_index=s.reset_index() #we should save the index if we are to merge 1/0 to these values, and if there are some NaN
s_b=s_with_index.merge(b_gr, 
            how='left', 
            left_on=['Date','Origin','Destination'], 
            right_on=['act_date', 'dep_port','arr_port'])

s_b.drop(['dep_port','arr_port', 'act_date'],axis=1, inplace=True)
s_b[s_b['Num of Bookings']>0].head()
s_b.tail()

,index,Date,Origin,Destination,Num of Bookings
717997,717998,2013-05-03,CPH,AYT,NaN
717998,717999,2013-05-03,NYC,YTO,NaN
717999,718000,2013-05-03,TRF,TRD,NaN
718000,718001,2013-05-03,IAH,RDU,NaN
718001,718002,2013-10-13,VIE,HA,NaN


In [95]:
s_with_index.rename(columns={'index':'old_index'}, inplace=True)

In [96]:
s_with_index.index[0]

0

In [97]:
s_with_index[s_with_index['old_index']!=s_with_index.index][0:10]

,old_index,Date,Origin,Destination
594999,595000,2013-01-01,ICT,SFO
595000,595001,2013-01-01,RNB,ARN
595001,595002,2013-01-01,OSL,MAD
595002,595003,2013-01-01,IAH,BLR
595003,595004,2013-01-01,CPH,PAR
595004,595005,2013-01-01,PAR,DUB
595005,595006,2013-01-01,DUS,ACE
595006,595007,2013-01-01,FRA,BGW
595007,595008,2013-01-01,TXL,AUH
595008,595009,2013-01-01,ATH,MIL


In [98]:
s_with_index.tail()

,old_index,Date,Origin,Destination
717997,717998,2013-05-03,CPH,AYT
717998,717999,2013-05-03,NYC,YTO
717999,718000,2013-05-03,TRF,TRD
718000,718001,2013-05-03,IAH,RDU
718001,718002,2013-10-13,VIE,HA


#### If we dont want to drop the duplicates

In [99]:
b_gr2=b_gr.rename(columns={0:'Num of Bookings', 'act_date':'Date', 'dep_port':'Origin','arr_port':'Destination'})
s_b=s_with_index.merge(b_gr2, 
            how='left')
s_b[s_b['Num of Bookings']>0].head()

,old_index,Date,Origin,Destination,Num of Bookings
23,23,2013-01-01,DUS,IST,1.0
27,27,2013-01-01,RUH,JED,17.0
40,40,2013-01-01,DMM,MNL,1.0
59,59,2013-01-01,ATL,MIA,4.0
94,94,2013-01-01,DXB,KUL,1.0


In [100]:
s_b.tail()

,old_index,Date,Origin,Destination,Num of Bookings
717997,717998,2013-05-03,CPH,AYT,NaN
717998,717999,2013-05-03,NYC,YTO,NaN
717999,718000,2013-05-03,TRF,TRD,NaN
718000,718001,2013-05-03,IAH,RDU,NaN
718001,718002,2013-10-13,VIE,HA,NaN


In [101]:
len(s_b), len(s)

(718002, 718002)

### Now we have to put 1 to all the booking with Num of Bookings>1 in the clean file

In [16]:
s_all=pd.read_csv(file_names,sep='^', low_memory=False)
s_all.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


In [76]:
# s_all=s_all.dropna(subset = ['Date', 'Origin','Destination'])

In [77]:
len(s_all.dropna(subset = ['Date', 'Origin','Destination']))

718002

In [17]:
len(s_all),len(s_b)

(718003, 718002)

In [20]:
result=s_all.merge(s_b, 
            how='left', 
            left_index=True,
            right_on='old_index')

In [21]:
result[result['Num of Bookings']>1].tail(10)

,Date_x,Time,TxnCode,OfficeID,Country,Origin_x,Destination_x,RoundTrip,NbSegments,Seg1Departure,...,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,old_index,Date_y,Origin_y,Destination_y,Num of Bookings
717831,2013-05-03,01:33:05,MPT,8b4fa4e121637f583bfa86ef0fa26c9e,US,LAX,LAS,1.0,2.0,LAX,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LAS,717832,2013-05-03,LAX,LAS,2.0
717856,2013-05-03,13:59:20,MPT,440642a9bdaeb6287f826cefd73255e8,US,KCH,KUL,1.0,2.0,KCH,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HPN,717857,2013-05-03,KCH,KUL,2.0
717859,2013-05-03,21:04:28,MPT,f976842f00052a55080c5ea015cd728c,US,JFK,SDQ,1.0,2.0,JFK,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SFO,717860,2013-05-03,JFK,SDQ,2.0
717885,2013-05-03,11:45:21,MPT,cf4ae41b25a075c3a8fee145e1919199,NL,AMS,CDG,1.0,2.0,AMS,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,AMS,717886,2013-05-03,AMS,CDG,4.0
717892,2013-05-03,17:17:18,FXX,1722290df1e162d326280c5d4a82657b,CO,BAQ,BOG,0.0,1.0,BAQ,...,0,0,0,c01f2026c4c0064e4a406973f52e1375,BOG,717893,2013-05-03,BAQ,BOG,4.0
717914,2013-05-03,16:00:14,FXL,9c7e0c8dd5b434e375f293130d54aa98,CO,MDE,BOG,1.0,2.0,MDE,...,0,0,0,40b17e34908fcd5f252369ea4aa546c7,CLO,717915,2013-05-03,MDE,BOG,8.0
717926,2013-05-03,14:34:47,MPT,d1b80a3dca253a3816073469f2cfe6be,GB,LHR,MIA,1.0,2.0,LHR,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LON,717927,2013-05-03,LHR,MIA,2.0
717943,2013-05-03,15:46:07,MTC,7f756d89441bf77ca6cfc2c69e358aa8,ES,MAD,SFO,0.0,1.0,MAD,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,MAD,717944,2013-05-03,MAD,SFO,4.0
717964,2013-05-03,04:37:54,AWT,ccb20b39ccfeccfe2cf6ee59fe243093,FR,CDG,LAX,0.0,1.0,CDG,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,CDG,717965,2013-05-03,CDG,LAX,2.0
717966,2013-05-03,18:59:04,CAL,c500a9ba3f7a21c286fdefc5c8c2ff99,BG,SOF,PMI,1.0,2.0,SOF,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SOF,717967,2013-05-03,SOF,PMI,2.0


In [23]:
result.tail()

,Date_x,Time,TxnCode,OfficeID,Country,Origin_x,Destination_x,RoundTrip,NbSegments,Seg1Departure,...,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,old_index,Date_y,Origin_y,Destination_y,Num of Bookings
717997,2013-05-03,10:28:25,MPT,01a4373dc0878961ab038a4089f8d4a7,DK,CPH,AYT,1.0,2.0,CPH,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,CPH,717998,2013-05-03,CPH,AYT,NaN
717998,2013-05-03,15:00:52,MPT,4e93208c7efe7d16455c6a887367cdbd,US,NYC,YTO,0.0,1.0,NYC,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,DCA,717999,2013-05-03,NYC,YTO,NaN
717999,2013-05-03,00:40:36,FXP,439c0bb10ec8b58c68da567c40399a05,NO,TRF,TRD,0.0,1.0,TRF,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,718000,2013-05-03,TRF,TRD,NaN
718000,2013-05-03,21:10:30,MPT,3561a60621de06ab1badc8ca55699ef3,US,IAH,RDU,1.0,2.0,IAH,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,718001,2013-05-03,IAH,RDU,NaN
718001,2013-10-13,18:57:54,MTP,e41c9d833aa74600552f2ed688b67d81,AT,VIE,HA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,718002,2013-10-13,VIE,HA,NaN


In [25]:
s_all.tail()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
717998,2013-05-03,10:28:25,MPT,01a4373dc0878961ab038a4089f8d4a7,DK,CPH,AYT,1.0,2.0,CPH,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,CPH
717999,2013-05-03,15:00:52,MPT,4e93208c7efe7d16455c6a887367cdbd,US,NYC,YTO,0.0,1.0,NYC,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,DCA
718000,2013-05-03,00:40:36,FXP,439c0bb10ec8b58c68da567c40399a05,NO,TRF,TRD,0.0,1.0,TRF,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL
718001,2013-05-03,21:10:30,MPT,3561a60621de06ab1badc8ca55699ef3,US,IAH,RDU,1.0,2.0,IAH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
718002,2013-10-13,18:57:54,MTP,e41c9d833aa74600552f2ed688b67d81,AT,VIE,HA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
len(result)

718003

In [27]:
result.ix[718001]

,Date_x,Time,TxnCode,OfficeID,Country,Origin_x,Destination_x,RoundTrip,NbSegments,Seg1Departure,...,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,old_index,Date_y,Origin_y,Destination_y,Num of Bookings
718001,"2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,594999,NaT,NaN,NaN,NaN
718001,2013-10-13,18:57:54,MTP,e41c9d833aa74600552f2ed688b67d81,AT,VIE,HA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,718002,2013-10-13,VIE,HA,NaN


In [29]:
result.iloc[594999]

Date_x               2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5c...
Time                                                               NaN
TxnCode                                                            NaN
OfficeID                                                           NaN
Country                                                            NaN
Origin_x                                                           NaN
Destination_x                                                      NaN
RoundTrip                                                          NaN
NbSegments                                                         NaN
Seg1Departure                                                      NaN
Seg1Arrival                                                        NaN
Seg1Date                                                           NaN
Seg1Carrier                                                        NaN
Seg1BookingCode                                                    NaN
Seg2De

In [30]:
result.drop(['Date_y','Origin_y','Destination_y'], axis=1, inplace=True)
result.rename(columns={'Date_x':'Date','Num of Bookings':'Booked'}, inplace=True)
result['Booked']=result['Booked'].fillna(0).map(lambda x: 1 if x>0 else 0)

In [29]:
result.to_csv('searches_bookings.csv',sep='^', index=False)

,Date,Time,TxnCode,OfficeID,Country,Origin_x,Destination_x,RoundTrip,NbSegments,Seg1Departure,...,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,index,Booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0,0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,1,0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,2,0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,...,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,3,0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,4,0


### Can we do it with concat?
no, since it just joins on the index

In [30]:
pd.concat?

In [20]:
result2=pd.concat([s_all, s_b], axis=1)
#result2=pd.concat([s_all, s_b['Num of Bookings']], axis=1)
#result2.head()

In [21]:
result2.tail()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,index,Date,Origin,Destination,Num of Bookings
717998,2013-05-03,10:28:25,MPT,01a4373dc0878961ab038a4089f8d4a7,DK,CPH,AYT,1,2,CPH,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,CPH,717999,2013-05-03,NYC,YTO,NaN
717999,2013-05-03,15:00:52,MPT,4e93208c7efe7d16455c6a887367cdbd,US,NYC,YTO,0,1,NYC,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,DCA,718000,2013-05-03,TRF,TRD,NaN
718000,2013-05-03,00:40:36,FXP,439c0bb10ec8b58c68da567c40399a05,NO,TRF,TRD,0,1,TRF,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,718001,2013-05-03,IAH,RDU,NaN
718001,2013-05-03,21:10:30,MPT,3561a60621de06ab1badc8ca55699ef3,US,IAH,RDU,1,2,IAH,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,718002,2013-10-13,VIE,HA,NaN
718002,2013-10-13,18:57:54,MTP,e41c9d833aa74600552f2ed688b67d81,AT,VIE,HA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [22]:
#result2[result2['Num of Bookings']>1].tail(10)

In [76]:
result.Booked.fillna(value=0, inplace=True)
result.head()

,Booked,Country,Date,Destination,From,InternetOffice,IsFromInternet,IsFromVista,IsPublishedForNeg,NbSegments,...,Seg5Date,Seg5Departure,Seg6Arrival,Seg6BookingCode,Seg6Carrier,Seg6Date,Seg6Departure,TerminalID,Time,TxnCode
0,0,DE,2013-01-01,AUH,1ASIWS,FRA,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20:25:57,MPT
1,0,MD,2013-01-01,MIL,1ASIWS,KIV,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,10:15:33,MPT
2,0,US,2013-01-01,SFO,1ASIWS,NYC,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,18:04:49,MPT
3,0,SE,2013-01-01,ARN,1ASI,STO,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,17:42:40,FXP
4,0,NO,2013-01-01,MAD,1ASIWS,OSL,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,17:48:29,MPT


### Check if all airports are letters

In [82]:
int(s.ix[317322]['Origin'])

240

In [84]:
def check_airports (airport):
    try:
        int(airport)
        print "%s is not airport"%(airport)
        return False
    except:
        return True

In [90]:
int('012')

12

In [85]:
b=b.reset_index()
b.head()

,index,act_date,dep_port,arr_port
0,0,2013-03-05,ZRH,LHR
1,1,2013-03-26,SAL,CLT
2,2,2013-03-26,SAL,CLT
3,3,2013-03-26,AKL,SVO
4,4,2013-03-26,AKL,SVO


In [86]:
check_airports('123')

123 is not airport


False

In [87]:
s.head().Origin.map(check_airports)

0    True
1    True
2    True
3    True
4    True
Name: Origin, dtype: bool

In [206]:
s_test=s.head()

s_test2 = s[s.Origin.map(check_airports)]
s_test2

240 is not airport


,Date,Origin,Destination
0,2013-01-01,TXL,AUH
1,2013-01-01,ATH,MIL
2,2013-01-01,ICT,SFO
3,2013-01-01,RNB,ARN
4,2013-01-01,OSL,MAD
5,2013-01-01,IAH,BLR
6,2013-01-01,CPH,PAR
7,2013-01-01,PAR,DUB
8,2013-01-01,DUS,ACE
9,2013-01-01,FRA,BGW


In [88]:
print "before airport check", len(s)
s = s[s.Origin.map(check_airports)]
s = s[s.Destination.map(check_airports)]
print "after airport check", len(s)

240 is not airport
240 is not airport


718000

### izbaceni brojevi

In [111]:
b.sort_values(['dep_port','arr_port', 'act_date'],inplace=True)

In [112]:
print s.head(10)
print b.head(10)

        search ID       Date Origin Destination
49789       49789 2013-02-19    AAC         CAI
170497     170497 2013-06-20    AAE         ALG
141836     141836 2013-05-22    AAE         MRS
231815     231816 2013-08-20    AAE         NCE
278500     278501 2013-10-06    AAE         ORY
181303     181303 2013-07-01    AAE         PAR
234611     234612 2013-08-23    AAE         PAR
99103       99103 2013-04-10    AAL         AGP
169509     169509 2013-06-19    AAL         AGP
184042     184042 2013-07-04    AAL         AGP
         book ID   act_date dep_port arr_port
387078    387078 2013-05-29      AAB      ROV
387079    387079 2013-05-29      AAB      ROV
1387081   387088 2013-05-29      AAB      ROV
1387082   387089 2013-05-29      AAB      ROV
572522    572522 2013-08-24      AAB      ROV
572523    572523 2013-08-24      AAB      ROV
1572525   572532 2013-08-24      AAB      ROV
1572526   572533 2013-08-24      AAB      ROV
165186    165186 2013-01-07      AAE      ALG
1165189   16

In [74]:
b.rename(columns={'index':'book ID'}, inplace=True)
b.head()

,book ID,act_date,dep_port,arr_port
0,0,2013-03-05,ZRH,LHR
1,1,2013-03-26,SAL,CLT
2,2,2013-03-26,SAL,CLT
3,3,2013-03-26,AKL,SVO
4,4,2013-03-26,AKL,SVO


In [75]:
s=s.reset_index()
s.rename(columns={'index':'search ID'}, inplace=True)
s.head()

In [76]:
s.head()

,search ID,Date,Origin,Destination
0,0,2013-01-01,TXL,AUH
1,1,2013-01-01,ATH,MIL
2,2,2013-01-01,ICT,SFO
3,3,2013-01-01,RNB,ARN
4,4,2013-01-01,OSL,MAD


In [78]:
s.sort_values(by=['Origin','Destination', 'Date'],inplace=True)
s.head(10)

,search ID,Date,Origin,Destination
317322,317323,2013-11-14,240,INR
49789,49789,2013-02-19,AAC,CAI
170497,170497,2013-06-20,AAE,ALG
141836,141836,2013-05-22,AAE,MRS
231815,231816,2013-08-20,AAE,NCE
278500,278501,2013-10-06,AAE,ORY
181303,181303,2013-07-01,AAE,PAR
234611,234612,2013-08-23,AAE,PAR
99103,99103,2013-04-10,AAL,AGP
169509,169509,2013-06-19,AAL,AGP
